### # Homework 1

In [179]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date, datetime, timedelta


## Question 1: [Macro] Average growth of GDP in 2023

What is the average growth (in %) of GDP in 2023?

Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1). Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago). Find the average YoY growth in 2023 (average from 4 YoY numbers). Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer 5.7

In [8]:
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=end.year-70, month=end.month, day=end.day)
print(f'Period for indexes: {start} to {end} ')

Year = 2024; month= 4; day=19
Period for indexes: 1954-04-19 to 2024-04-19 


In [9]:
# Real Gross Domestic Product (GDPC1) 
# https://fred.stlouisfed.org/series/GDPC1
gdpc1 = pdr.DataReader("GDPC1", "fred", start=start)
gdpc1.tail(5)


,GDPC1
DATE,
2022-10-01,21989.981
2023-01-01,22112.329
2023-04-01,22225.350
2023-07-01,22490.692
2023-10-01,22679.255


In [10]:
gdpc1['gdpc1_us_yoy'] = gdpc1.GDPC1/gdpc1.GDPC1.shift(4)-1
mean_gdpc1 = gdpc1['gdpc1_us_yoy'][-4:].mean()*100
print(f"The mean real GDP growth YoY for the us is {mean_gdpc1:2.1f}%")

The mean real GDP growth YoY for the us is 2.5%


## Question 2. [Macro] Inverse "Treasury Yield"

Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.

Download DGS2 and DGS10 interest rates series (https://fred.stlouisfed.org/series/DGS2, https://fred.stlouisfed.org/series/DGS10). Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join()), calculate the difference dgs10-dgs2 daily.

(Additional: think about what does the "inverted yield curve" mean for the market and investors? do you see the same thing in your country/market of interest? Do you think it can be a good predictive feature for the models?)

In [29]:
start = date(year=2000, month=1, day=1)

dgs = pdr.DataReader(['dgs2', 'dgs10'], "fred", start=start)
dgs.tail(5)
dgs['inv'] = dgs.dgs10 - dgs.dgs2
print(f'The maximum inversion was on {dgs.inv.idxmin().date()} at {dgs.inv.min():.1f}')

The maximum inversion was on 2023-07-03 at -1.1


This means borrowing money on a shorter term is more expensive. Usually, the yield of longer term debt is higher, because investors don't have the same "flexibility" compared to the shorter term contract.
This can be a very good predictor for economic growth (as shown in this old [paper](https://users.ssc.wisc.edu/~mchinn/Harvey_yieldcurve_FAJ1989.pdf)), which certainly influences the valuation of stock prices.

## Question 3. [Index] Which Index is better recently?

Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)

Download on Yahoo Finance two daily index prices for S&P 500 (^GSPC, https://finance.yahoo.com/quote/%5EGSPC/) and IPC Mexico (^MXX, https://finance.yahoo.com/quote/%5EMXX/). Compare 5Y growth for both (between 2019-04-09 and 2024-04-09). Select the higher growing index and write down the growth in % (closest integer %). E.g. if ratio end/start was 2.0925 (or growth of 109.25%), you need to write down 109 as your answer.

(Additional: think of other indexes and try to download stats and compare the growth? Do create 10Y and 20Y growth stats. What is an average yearly growth rate (CAGR) for each of the indexes you select?)

In [34]:
start="2019-04-09"
end="2024-04-09"

index_daily = yf.download(tickers = ["^GSPC", "^MXX"],
                     start=start,
                     end=end,
                     interval = "1d")

[*********************100%%**********************]  2 of 2 completed


In [68]:
mean_growth = (index_daily.Close.iloc[-1] / index_daily.Open.iloc[0] -1)*100

print(f"The growth for the S&P 500 between {start} and {end} is {mean_growth['^GSPC']:.0f}%.")
print(f"The growth for the IPC Mexico  between {start} and {end} is {mean_growth['^MXX']:.0f}%.\n")

cagr = ((index_daily.Close.iloc[-1] / index_daily.Open.iloc[0])**(1/5) -1)*100

print(f"The cagr for the S&P 500 between {start} and {end} is {cagr['^GSPC']:.0f}%.")
print(f"The cagr for the IPC Mexico  between {start} and {end} is {cagr['^MXX']:.0f}%.")

The growth for the S&P 500 between 2019-04-09 and 2024-04-09 is 80%.
The growth for the IPC Mexico  between 2019-04-09 and 2024-04-09 is 28%.

The cagr for the S&P 500 between 2019-04-09 and 2024-04-09 is 13%.
The cagr for the IPC Mexico  between 2019-04-09 and 2024-04-09 is 5%.


## Question 4. [Stocks OHLCV] 52-weeks range ratio (2023) for the selected stocks

Find the largest range ratio [=(max-min)/max] of Adj.Close prices in 2023

Download the 2023 daily OHLCV data on Yahoo Finance for top6 stocks on earnings (https://companiesmarketcap.com/most-profitable-companies/): 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM.

Here is the example data you should see in Pandas for "2222.SR": https://finance.yahoo.com/quote/2222.SR/history

Calculate maximum-minimim "Adj.Close" price for each stock and divide it by the maximum "Adj.Close" value. Round the result to two decimal places (e.g. 0.1575 will be 0.16)

(Additional: why this may be important for your research?)

In [131]:
tickers = ['2222.SR','BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']

start="2023-01-01"
end="2023-12-31"

stocks = yf.download(tickers = tickers,
                     start=start,
                     end=end,
                     interval = "1d")

[*********************100%%**********************]  6 of 6 completed


In [86]:
rel_stocks = (stocks['Adj Close'] - stocks['Adj Close'].min(axis=0)) / stocks['Adj Close'].max(axis=0)

rel_stocks.max(axis=0).round(2)

Ticker
2222.SR    0.21
AAPL       0.37
BRK-B      0.21
GOOG       0.39
JPM        0.28
MSFT       0.42
dtype: float64

This metric gives a feeling for the appreciation of the stock in the given timeframe. E.g. if a stock from the same sector did not appreciate by the same amount compeared to the mean of the sector, this stock would be "underperforming". There might me a reason, there might not be.

## Question 5. [Stocks] Dividend Yield

Find the largest dividend yield for the same set of stocks

Use the same list of companies (2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM) and download all dividends paid in 2023. You can use get_actions() method or .dividends field in yfinance library (https://github.com/ranaroussi/yfinance?tab=readme-ov-file#quick-start)

Sum up all dividends paid in 2023 per company and divide each value by the closing price (Adj.Close) at the last trading day of the year.

Find the maximm value in % and round to 1 digit after the decimal point. (E.g., if you obtained $1.25 dividends paid and the end year stock price is $100, the dividend yield is 1.25% -- and your answer should be equal to 1.3)

In [128]:
tickers = ['2222.SR','BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']

#Define the period for which you want the dividend data
start_date = "2023-01-01"
end_date = "2023-12-31"

# Initialize Tickers object with all tickers
stocks = yf.Tickers(" ".join(tickers))

# Create an empty DataFrame to store dividend data
dividends_df = pd.DataFrame()

# Download and extract dividend data for each ticker
for ticker in tickers:
    dividends = stocks.tickers[ticker].dividends.loc[start_date:end_date]
    # Ensure dividends are labeled by their payment date and ticker
    dividends.name = ticker
    # Join each new Series to the existing DataFrame, aligning on dates
    dividends_df = pd.concat([dividends_df, dividends], axis=1)

# Fill NaN values with 0, assuming no dividend was paid on that date
dividends_df.fillna(0, inplace=True)
dividends_df.sort_index(inplace=True)

# Display the resulting DataFrame
print(dividends_df)

                            2222.SR  BRK-B  AAPL  MSFT  GOOG   JPM
2023-01-05 00:00:00-05:00  0.000000    0.0  0.00  0.00   0.0  1.00
2023-02-10 00:00:00-05:00  0.000000    0.0  0.23  0.00   0.0  0.00
2023-02-15 00:00:00-05:00  0.000000    0.0  0.00  0.68   0.0  0.00
2023-03-15 00:00:00+03:00  0.302364    0.0  0.00  0.00   0.0  0.00
2023-04-05 00:00:00-04:00  0.000000    0.0  0.00  0.00   0.0  1.00
2023-05-12 00:00:00-04:00  0.000000    0.0  0.24  0.00   0.0  0.00
2023-05-17 00:00:00+03:00  0.302400    0.0  0.00  0.00   0.0  0.00
2023-05-17 00:00:00-04:00  0.000000    0.0  0.00  0.68   0.0  0.00
2023-07-05 00:00:00-04:00  0.000000    0.0  0.00  0.00   0.0  1.00
2023-08-11 00:00:00-04:00  0.000000    0.0  0.24  0.00   0.0  0.00
2023-08-16 00:00:00-04:00  0.000000    0.0  0.00  0.68   0.0  0.00
2023-09-11 00:00:00+03:00  0.153000    0.0  0.00  0.00   0.0  0.00
2023-10-05 00:00:00-04:00  0.000000    0.0  0.00  0.00   0.0  1.05
2023-11-10 00:00:00-05:00  0.000000    0.0  0.24  0.00   0.0  

In [176]:
# Retrieve the last non-NaN value from each column and form a new DataFrame
def last_valid_in_df(dataframe):
    last_valid = {}
    for column in dataframe.columns:
        # Drop NaN values from the column and retrieve the last valid value
        cleaned_series = dataframe[column].dropna()
        if not cleaned_series.empty:
            last_valid[column] = cleaned_series.iloc[-1]
        else:
            last_valid[column] = np.nan  # Handling the case where the column is all NaN

    # Convert the dictionary to a DataFrame
    result_df = pd.DataFrame([last_valid])  # Note the [last_valid] to make it a row in the DataFrame
    return result_df

# Apply the function to the DataFrame
last_valid_df = last_valid_in_df(stocks.Close)

div_yield = dividends_df.sum(axis=0) / last_valid_df * 100

for column in div_yield.columns:
    print(f"The dividend yield for {column} is {div_yield[column].iloc[0]:.1f}%")


The dividend yield for 2222.SR is 2.8%
The dividend yield for AAPL is 0.5%
The dividend yield for BRK-B is 0.0%
The dividend yield for GOOG is 0.0%
The dividend yield for JPM is 2.4%
The dividend yield for MSFT is 0.7%


## Question 6. [Exploratory] Investigate new metrics

Free text answer

Download and explore a few additional metrics or time series that might be valuable for your project and write down why (briefly).

check the correlation of different sectors based on different market regimes (CPI, interest rates, Real GDP) & Analyse future data (roll yield on different commodities with possible hedges)

## Question 7. [Exploratory] Time-driven strategy description around earnings releases

Free text answer

Explore earning dates for the whole month of April - e.g. using YahooFinance earnings calendar (https://finance.yahoo.com/calendar/earnings?from=2024-04-21&to=2024-04-27&day=2024-04-23). Compare with the previous closed earnings (e.g., recent dates with full data https://finance.yahoo.com/calendar/earnings?from=2024-04-07&to=2024-04-13&day=2024-04-08).

Describe an analytical strategy/idea (you're not required to implement it) to select a subset companies of interest based on the future events data.

If a company of a sector reports strong earning, it might be likely another company from the same sector might also post good results.